In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
files = ['2010_Data.xlsx', '2011_Data.xlsx', '2012_Data.xlsx', '2013_Data.xlsx', '2014_Data.xlsx']
df = pd.read_excel('2009_Data.xlsx')
for file in files:
    df = df.append(pd.read_excel(file))

In [3]:
df = df[df['vnumber'] == 1].copy()

In [ ]:
df

In [4]:
df_names = pd.read_excel('Uber_Data.xlsx')

In [5]:
df.drop(['pnumber', 'race', 'alcres', 'ptype'], axis=1, inplace=True)
df.rename(columns={'caseyear': 'Year', 'accmon': 'Month', 'county': 'countynum'}, inplace=True)
df['Date'] = pd.to_datetime(df[['Year', 'Month']].assign(Day=1))
df_names['DateTreat'] = pd.to_datetime(df_names[['Year', 'Month']].assign(Day=1))
df_names.rename(columns={'Year': 'YearTreat', 'Month': 'MonthTreat', 'County Code': 'countynum', 'State Code': 'statenum'}, inplace=True)
df_names.drop(['Metropolitan Statistical Area'], axis=1, inplace=True)

In [ ]:
# county_name = ['Jefferson', 'Maricopa', 'Pima','Pulaski','Fresno','Kern','Los Angeles','Riverside','Sacramento','San Diego','San Francisco','San Joaquin','Santa Clara'
# ,'Ventura','Denver','El Paso','Fairfield','Hartford','New Haven','District of Columbia','Brevard','Duval','Hillsborough','Lee','Miami-Dade','Orange','Polk'
# ,'Sarasota','Volusia','Fulton','Richmond','Honolulu','Ada','Cook','Marion','Polk','Sedgwick','Jefferson','East Baton Rouge','Orleans','Baltimore','Hampden'
# ,'Suffolk','Worcester','Kent','Wayne','Hennepin','Hinds','Jackson','St. Louis','Douglas','Clark','Bernalillo','Albany','Bronx','Erie,Kings','Monroe','New York'
# ,'Onondaga','Queens','Richmond','Durham','Forsyth','Guilford','Mecklenburg','Wake','Cuyahoga','Franklin','Hamilton','Lucas','Mahoning','Montgomery','Summit'
# ,'Oklahoma','Tulsa','Multnomah','Allegheny','Dauphin','Lackawanna','Lehigh','Philadelphia','Providence','Charleston','Greenville','Richland','Davidson'
# ,'Hamilton','Knox','Shelby','Bexar','Dallas','El Paso','Harris','Hidalgo','Travis','Salt Lake','Utah','Weber','Richmond','Virginia Beach','King','Dane','Milwaukee']


In [ ]:
df

In [6]:
state_name = list(df_names.State)
state_code = list(df_names['statenum'])
county_name = list(df_names.County)
county_code = list(df_names['countynum'])
pop = list(df_names['population'])
date = list(df_names['DateTreat'])

In [7]:
c = []
s = []

for s_code, c_code, treat in zip(state_code, county_code, date):
    c0 = ((df.statenum == s_code) & (df.countynum == c_code))
    c.append(c0)
    s0 = (df.statenum == s_code)
    s.append(s0)
    
    
df['County'] = np.select(c, county_name, default='drop')
df['State'] = np.select(s, state_name, default='drop')

In [8]:
df = df[(df.County != 'drop')].copy()
df_bmi = df[(df.County != 'drop')].copy()

In [9]:
c0 = ((df.alcinvol == 1) | (df.malcohol == 1))

df['Alcohol'] = np.select([c0], ['alcohol'], default='no alcohol')
df_bmi['Alcohol'] = np.select([c0], ['alcohol'], default='no alcohol')

In [10]:
df_bmi = df_bmi[(df_bmi.feet != '.')].copy()
df_bmi['feet'] = df_bmi['feet'].apply(int)
df_bmi['inches'] = df_bmi['inches'].apply(int)
df_bmi['weight'] = df_bmi['weight'].apply(int)
df_bmi = df_bmi[df_bmi.weight <= 900]

In [11]:
df_bmi['Meters'] = ((df_bmi['feet']*12 + df_bmi['inches']) / 39.37)
df_bmi['Kilo'] = (df_bmi['weight'] / 2.205)
df_bmi['BMI'] = (df_bmi['Kilo'] / (df_bmi['Meters'] ** 2))

In [12]:
idx = pd.date_range('01-01-2009','12-01-2014',freq='MS')
s_n = len(idx)
df_n = len(df_names)
merge1 = pd.DataFrame(np.repeat(df_names.values,s_n, axis = 0), columns = df_names.columns, index = np.tile(idx,df_n)).rename_axis('Date').reset_index()
merge1['statenum']=merge1['statenum'].apply(int)
merge1['countynum']=merge1['countynum'].apply(int)
merge1.drop(['MonthTreat', 'YearTreat','DateTreat'], axis=1, inplace = True)
merge1.head()

,Date,State,County,population,countynum,statenum,Treated,Code
0,2009-01-01,Alabama,Jefferson,1129034,73,1,0,1
1,2009-02-01,Alabama,Jefferson,1129034,73,1,0,1
2,2009-03-01,Alabama,Jefferson,1129034,73,1,0,1
3,2009-04-01,Alabama,Jefferson,1129034,73,1,0,1
4,2009-05-01,Alabama,Jefferson,1129034,73,1,0,1


In [ ]:
merge2 = df.groupby(['statenum','countynum','Date','Year','Month']).agg(
    {'casenum':'count', 'numfatal': 'sum'}).rename(columns={'casenum':'numcrashes'}).reset_index()
merge2.head()

In [ ]:
df.head()

In [ ]:
final = pd.merge(merge1, merge2, on=["Date", "statenum", "countynum"], how="left")

In [ ]:
# final[['numcrashes', 'numfatal']] = final[['numcrashes', 'numfatal']].fillna(value=0)
# df1[['statenum','countynum']] = df1[['statenum','countynum']].fillna(method='ffill')

In [ ]:
c = []
t = []
tr = []

for s_code, c_code, treat in zip(state_code, county_code, date):
    c0 = ((final.statenum == s_code) & (final.countynum == c_code))
    c.append(c0)
    t0 = ((final.statenum == s_code) & (final.countynum == c_code) & (final.Date >= treat))
    t.append(t0)
    t1 = ((final.statenum == s_code) & (final.countynum == c_code) & (final.Treated == 1))
    tr.append(t1)
    

final['Population'] = np.select(c, pop, default=np.nan)
final['Treatment'] = np.select(t, [1]*len(t), default=0)
final['Treated'] = np.select(tr, [1]*len(tr), default=0)

In [ ]:
final[['numcrashes', 'numfatal']] = final[['numcrashes', 'numfatal']].fillna(value=0)
final['Year'] = final['Date'].dt.year
final['Month'] = final['Date'].dt.month
final.head()

In [ ]:
final.to_excel('testing5.xlsx', sheet_name='total')

In [31]:
merge2 = df.groupby(['age_group','sex','Alcohol','statenum','countynum','Date','Year','Month']).agg(
    {'casenum':'count', 'numfatal': 'sum'}).rename(columns={'casenum':'numcrashes'}).reset_index()
merge2 = merge2[((merge2['Alcohol'] == 'alcohol') & (merge2['sex'] == 1) & (merge2['age_group'] == 2))]

In [32]:
final = pd.merge(merge1, merge2, on=["Date", "statenum", "countynum"], how="left")

In [33]:
c = []
t = []
tr = []

for s_code, c_code, treat in zip(state_code, county_code, date):
    c0 = ((final.statenum == s_code) & (final.countynum == c_code))
    c.append(c0)
    t0 = ((final.statenum == s_code) & (final.countynum == c_code) & (final.Date >= treat))
    t.append(t0)
    t1 = ((final.statenum == s_code) & (final.countynum == c_code) & (final.Treated == 1))
    tr.append(t1)
    

final['Population'] = np.select(c, pop, default=np.nan)
final['Treatment'] = np.select(t, [1]*len(t), default=0)
final['Treated'] = np.select(tr, [1]*len(tr), default=0)

In [34]:
final[['numcrashes', 'numfatal']] = final[['numcrashes', 'numfatal']].fillna(value=0)
final['Alcohol'] = final['Alcohol'].fillna(method='ffill')
final['Year'] = final['Date'].dt.year
final['Month'] = final['Date'].dt.month
final
# final.to_excel('testing6.xlsx', sheet_name='alcohol')

,Date,State,County,population,countynum,statenum,Treated,Code,age_group,sex,Alcohol,Year,Month,numcrashes,numfatal,Population,Treatment
0,2009-01-01,Alabama,Jefferson,1129034,73,1,0,1,NaN,NaN,NaN,2009,1,0.0,0.0,1129034.0,0
1,2009-02-01,Alabama,Jefferson,1129034,73,1,0,1,NaN,NaN,NaN,2009,2,0.0,0.0,1129034.0,0
2,2009-03-01,Alabama,Jefferson,1129034,73,1,0,1,2.0,1.0,alcohol,2009,3,2.0,2.0,1129034.0,0
3,2009-04-01,Alabama,Jefferson,1129034,73,1,0,1,NaN,NaN,alcohol,2009,4,0.0,0.0,1129034.0,0
4,2009-05-01,Alabama,Jefferson,1129034,73,1,0,1,NaN,NaN,alcohol,2009,5,0.0,0.0,1129034.0,0
5,2009-06-01,Alabama,Jefferson,1129034,73,1,0,1,2.0,1.0,alcohol,2009,6,1.0,1.0,1129034.0,0
6,2009-07-01,Alabama,Jefferson,1129034,73,1,0,1,NaN,NaN,alcohol,2009,7,0.0,0.0,1129034.0,0
7,2009-08-01,Alabama,Jefferson,1129034,73,1,0,1,NaN,NaN,alcohol,2009,8,0.0,0.0,1129034.0,0
8,2009-09-01,Alabama,Jefferson,1129034,73,1,0,1,NaN,NaN,alcohol,2009,9,0.0,0.0,1129034.0,0
9,2009-10-01,Alabama,Jefferson,1129034,73,1,0,1,NaN,NaN,alcohol,2009,10,0.0,0.0,1129034.0,0


In [ ]:
final

In [ ]:
merge2 = df.groupby(['Alcohol','statenum','countynum','Date','Year','Month']).agg(
    {'casenum':'count', 'numfatal': 'sum'}).rename(columns={'casenum':'numcrashes'}).reset_index()
merge2 = merge2[merge2['Alcohol'] == 'no alcohol']

In [ ]:
final = pd.merge(merge1, merge2, on=["Date", "statenum", "countynum"], how="left")

In [ ]:
c = []
t = []
tr = []

for s_code, c_code, treat in zip(state_code, county_code, date):
    c0 = ((final.statenum == s_code) & (final.countynum == c_code))
    c.append(c0)
    t0 = ((final.statenum == s_code) & (final.countynum == c_code) & (final.Date >= treat))
    t.append(t0)
    t1 = ((final.statenum == s_code) & (final.countynum == c_code) & (final.Treated == 1))
    tr.append(t1)
    

final['Population'] = np.select(c, pop, default=np.nan)
final['Treatment'] = np.select(t, [1]*len(t), default=0)
final['Treated'] = np.select(tr, [1]*len(tr), default=0)

In [ ]:
final[['numcrashes', 'numfatal']] = final[['numcrashes', 'numfatal']].fillna(value=0)
final['Alcohol'] = final['Alcohol'].fillna(method='ffill')
final['Year'] = final['Date'].dt.year
final['Month'] = final['Date'].dt.month
final
final.to_excel('testing7.xlsx', sheet_name='no alcohol')

In [ ]:
df_bmi['bmi_factor'] = ''
df_bmi.loc[df_bmi.BMI < 18.5, 'bmi_factor'] = "Underweight"
df_bmi.loc[((df_bmi.BMI >= 18.5) & (df_bmi.BMI < 25)), 'bmi_factor'] = "Normal"
df_bmi.loc[((df_bmi.BMI >= 25) & (df_bmi.BMI < 30)), 'bmi_factor'] = "Overweight"
df_bmi.loc[df_bmi.BMI >= 30, 'bmi_factor'] = "Obese"

In [ ]:
merge2 = df_bmi.groupby(['bmi_factor','statenum','countynum','Date','Year','Month']).agg(
    {'casenum':'count', 'numfatal': 'sum'}).rename(columns={'casenum':'numcrashes'}).reset_index()
merge_norm = merge2[merge2['bmi_factor'] == 'Normal']

In [ ]:
final = pd.merge(merge1, merge_norm, on=["Date", "statenum", "countynum"], how="left")

In [ ]:
c = []
t = []
tr = []

for s_code, c_code, treat in zip(state_code, county_code, date):
    c0 = ((final.statenum == s_code) & (final.countynum == c_code))
    c.append(c0)
    t0 = ((final.statenum == s_code) & (final.countynum == c_code) & (final.Date >= treat))
    t.append(t0)
    t1 = ((final.statenum == s_code) & (final.countynum == c_code) & (final.Treated == 1))
    tr.append(t1)
    

final['Population'] = np.select(c, pop, default=np.nan)
final['Treatment'] = np.select(t, [1]*len(t), default=0)
final['Treated'] = np.select(tr, [1]*len(tr), default=0)

In [ ]:
final[['numcrashes', 'numfatal']] = final[['numcrashes', 'numfatal']].fillna(value=0)
final['bmi_factor'] = final['bmi_factor'].fillna(method='ffill')
final['Year'] = final['Date'].dt.year
final['Month'] = final['Date'].dt.month
final.to_excel('testing8.xlsx', sheet_name='Normal')

In [ ]:
merge2 = df_bmi.groupby(['bmi_factor','statenum','countynum','Date','Year','Month']).agg(
    {'casenum':'count', 'numfatal': 'sum'}).rename(columns={'casenum':'numcrashes'}).reset_index()
merge_ow = merge2[merge2['bmi_factor'] == 'Overweight']

In [ ]:
final = pd.merge(merge1, merge_ow, on=["Date", "statenum", "countynum"], how="left")

In [ ]:
c = []
t = []
tr = []

for s_code, c_code, treat in zip(state_code, county_code, date):
    c0 = ((final.statenum == s_code) & (final.countynum == c_code))
    c.append(c0)
    t0 = ((final.statenum == s_code) & (final.countynum == c_code) & (final.Date >= treat))
    t.append(t0)
    t1 = ((final.statenum == s_code) & (final.countynum == c_code) & (final.Treated == 1))
    tr.append(t1)
    

final['Population'] = np.select(c, pop, default=np.nan)
final['Treatment'] = np.select(t, [1]*len(t), default=0)
final['Treated'] = np.select(tr, [1]*len(tr), default=0)

In [ ]:
final[['numcrashes', 'numfatal']] = final[['numcrashes', 'numfatal']].fillna(value=0)
final['bmi_factor'] = final['bmi_factor'].fillna(method='ffill')
final['Year'] = final['Date'].dt.year
final['Month'] = final['Date'].dt.month
final.to_excel('testing9.xlsx', sheet_name='Overweight')

In [ ]:
merge2 = df_bmi.groupby(['bmi_factor','statenum','countynum','Date','Year','Month']).agg(
    {'casenum':'count', 'numfatal': 'sum'}).rename(columns={'casenum':'numcrashes'}).reset_index()
merge_ob = merge2[merge2['bmi_factor'] == 'Obese']

In [ ]:
final = pd.merge(merge1, merge_ob, on=["Date", "statenum", "countynum"], how="left")

In [ ]:
c = []
t = []
tr = []

for s_code, c_code, treat in zip(state_code, county_code, date):
    c0 = ((final.statenum == s_code) & (final.countynum == c_code))
    c.append(c0)
    t0 = ((final.statenum == s_code) & (final.countynum == c_code) & (final.Date >= treat))
    t.append(t0)
    t1 = ((final.statenum == s_code) & (final.countynum == c_code) & (final.Treated == 1))
    tr.append(t1)
    

final['Population'] = np.select(c, pop, default=np.nan)
final['Treatment'] = np.select(t, [1]*len(t), default=0)
final['Treated'] = np.select(tr, [1]*len(tr), default=0)

In [ ]:
final[['numcrashes', 'numfatal']] = final[['numcrashes', 'numfatal']].fillna(value=0)
final['bmi_factor'] = final['bmi_factor'].fillna(method='ffill')
final['Year'] = final['Date'].dt.year
final['Month'] = final['Date'].dt.month
final.to_excel('testing10.xlsx', sheet_name='Obese')

In [ ]:
merge2 = df_bmi.groupby(['bmi_factor','statenum','countynum','Date','Year','Month']).agg(
    {'casenum':'count', 'numfatal': 'sum'}).rename(columns={'casenum':'numcrashes'}).reset_index()

merge_under = merge2[(merge2['bmi_factor'] == 'Underweight')]

In [ ]:
final = pd.merge(merge1, merge_under, on=["Date", "statenum", "countynum"], how="left")

In [ ]:
c = []
t = []
tr = []

for s_code, c_code, treat in zip(state_code, county_code, date):
    c0 = ((final.statenum == s_code) & (final.countynum == c_code))
    c.append(c0)
    t0 = ((final.statenum == s_code) & (final.countynum == c_code) & (final.Date >= treat))
    t.append(t0)
    t1 = ((final.statenum == s_code) & (final.countynum == c_code) & (final.Treated == 1))
    tr.append(t1)
    

final['Population'] = np.select(c, pop, default=np.nan)
final['Treatment'] = np.select(t, [1]*len(t), default=0)
final['Treated'] = np.select(tr, [1]*len(tr), default=0)

In [ ]:
final[['numcrashes', 'numfatal']] = final[['numcrashes', 'numfatal']].fillna(value=0)
final['bmi_factor'] = final['bmi_factor'].fillna(method='bfill')
final['Year'] = final['Date'].dt.year
final['Month'] = final['Date'].dt.month

In [ ]:
final.to_excel('testing11.xlsx')

In [ ]:
df = pd.read_excel('2009_Data.xlsx')

In [ ]:
df.county.isna().any()

In [ ]:
df[((df['statenum'] == 1) & (df['county']==73) & (df['accmon'] == 2))]

In [ ]:
df_names.head()

In [ ]:
df_names['State Code'] = df_names['State Code'].astype(str).str.zfill(2)

In [ ]:
df_names['County Code'] = df_names['County Code'].astype(str).str.zfill(3)

In [ ]:
df_names['Fips'] = df_names['State Code'] + df_names['County Code']

In [ ]:
df_names['DateTreat'] = pd.to_datetime(df_names[['Year', 'Month']].assign(Day=1))

In [ ]:
df_names['DateTreat'].fillna(method = 'bfill', inplace=True)

In [ ]:
import plotly.plotly as py
import plotly.figure_factory as ff

In [ ]:
fips = df_names['Fips']
values = df_names['Year']

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

colorscale = [
    'rgb(255, 0, 0)',
    'rgb(0,255,0)',
    'rgb(0, 0, 255)',
    'rgb(144,148,194)',
    'rgb(101,104,168)',
    'rgb(65, 53, 132)'
]

fig = ff.create_choropleth(fips=fips, values=values, state_outline={'width': 1}, county_outline={'color': 'rgb(255, 255, 255)', 'width': 0.1},height=720, width=1280, title='Uber Deployment Year', legend_title='Year')
plot(fig)

In [ ]:
import geopandas as pd

In [ ]:
df_names['Year'].fillna(0, inplace=True)

In [ ]:
len(values)

In [ ]:
df_names[['Fips', 'Year']]

In [24]:
bins = [0,15,20,25,30, 999]
labels=[0,1,2,3,4]
df['age_group'] = pd.cut(df['age'], bins=bins, labels=labels, include_lowest=True)

In [ ]:
df = df[df['sex'].isin([1,2])]

In [ ]:
df

In [15]:
merge2[((merge2['Alcohol'] == 'alcohol') & (merge2['sex'] == 1))]

,sex,Alcohol,statenum,countynum,Date,Year,Month,numcrashes,numfatal
0,1,alcohol,1,73,2009-02-01,2009,2,1,2
1,1,alcohol,1,73,2009-03-01,2009,3,2,2
2,1,alcohol,1,73,2009-04-01,2009,4,1,1
3,1,alcohol,1,73,2009-06-01,2009,6,2,2
4,1,alcohol,1,73,2009-07-01,2009,7,2,2
5,1,alcohol,1,73,2009-09-01,2009,9,1,1
6,1,alcohol,1,73,2009-10-01,2009,10,1,1
7,1,alcohol,1,73,2009-11-01,2009,11,3,3
8,1,alcohol,1,73,2010-02-01,2010,2,1,1
9,1,alcohol,1,73,2010-03-01,2010,3,1,1


In [35]:
final.to_excel('thing.xls')